In [1]:
import hopsworks
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib
import os

/home/redve/Kurser Master/ID2223 Scalable Machine Learning and Deep Learning/ID2223/Lab1/Task2/venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

for k, v in os.environ.items():
    if "HOPS" in k:
        print(f'{k}={v}')

HOPSWORKS_API_KEY=RJFCr0ixzSbcbtDM.awhoHRF08thPKMBz0x6BoX5piWGyY3DnGXfBKY3X4pCYQjxy6ypwNycQYbwPQgqj


In [3]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/190613
Connected. Call `.close()` to terminate connection gracefully.


In [10]:
wine_fg1 = fs.get_feature_group(name="winequality", version=15)
query1 = wine_fg1.select_all()
feature_view1 = fs.get_or_create_feature_view(name="winequality",
                                             version=15,
                                             description="Read from winequality feature group",
                                             labels=["quality"],
                                             query=query1)

# Read the data

In [27]:
x_train, x_test, y_train, y_test = feature_view1.train_test_split(test_size=0.2)

Finished: Reading data from Hopsworks, using ArrowFlight (10.32s) 


In [37]:
## Populate x_train['white'] if the value of type is 1 

for (idx, row) in x_train.iterrows():
    if row['type'] == 1:
        x_train.loc[idx, 'white'] = 1
        x_train.loc[idx, 'red'] = 0
    else:
        x_train.loc[idx, 'white'] = 0
        x_train.loc[idx, 'red'] = 1
for (idx,row) in x_test.iterrows():
    if row['type'] == 1:
        x_test.loc[idx, 'white'] = 1
        x_test.loc[idx, 'red'] = 0
    else:
        x_test.loc[idx, 'white'] = 0
        x_test.loc[idx, 'red'] = 1



In [43]:
train_set = pd.concat([x_train, y_train], axis=1)
train_set.head()
test_set = pd.concat([x_test, y_test], axis=1)
test_set.head()
train_set.drop(['type'], axis=1, inplace=True)
test_set.drop(['type'], axis=1, inplace=True)
train_set_y = train_set['quality']
train_set_x = train_set.drop(['quality'], axis=1)
test_set_y = test_set['quality']
test_set_x = test_set.drop(['quality'], axis=1)

In [47]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_set_x)
train_set_x = scaler.transform(train_set_x)
test_set_x = scaler.transform(test_set_x)


In [52]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
#creating RandomForestClassifier constructor
random_value = 42   
rnd = RandomForestClassifier(n_estimators=100, random_state=random_value)
fit_rnd = rnd.fit(train_set_x,train_set_y.values.ravel())
rnd_score = rnd.score(test_set_x,test_set_y.values.ravel())
print("Random Forest Classifier Score: ",rnd_score)

Random Forest Classifier Score:  0.5787992495309568


In [ ]:

mr = project.get_model_registry()


model_dir="wine_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

joblib.dump(model, model_dir + "/wine_model.pkl")
plt.savefig(model_dir + "/confusion_matrix.png")

input_schema = Schema(x_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

wine_model = mr.python.create_model(
    name="wine_model", 
    metrics={"accuracy" : score},
    model_schema=model_schema,
    description="Wine Quality Predictor"
)


wine_model.save(model_dir)
